In [4]:
%matplotlib widget
"""
Joshua Stough
DIP

Trying to display the Euclidean and the Haar basis transforms in simple 4x4.
Then, show a 4x4 image, its basis transform, and its reconstruction.
"""

import matplotlib.pyplot as plt
import numpy as np
import skimage
import scipy.ndimage as ndimage



#The 4x4 Euclidean basis. Each row represents an
#orthogonal (independent) direction in 4D space.
E = np.eye(4,4)

#For display
fe, axe = plt.subplots(4,4, figsize=(10,10))

for i in range(4):
    for j in range(4):
        #Taking the outer product of a row with any other
        #creates a 4x4 basis image.
        Bij = np.outer(E[i,:], E[j,:])
        axe[i][j].imshow(Bij, cmap='gray')
        axe[i][j].axes.get_xaxis().set_visible(False)
        axe[i][j].axes.get_yaxis().set_visible(False)

#plt.suptitle('Euclidean Basis') #gets in the way if using tight_layout(),
#see below.
plt.show()

#####################################################################

#The 4x4 Haar basis, see DIP 6.9.
H = .5*np.array([[1,1,1,1], [1,1,-1,-1],
              [np.sqrt(2),-np.sqrt(2),0,0],
              [0,0,np.sqrt(2),-np.sqrt(2)]])

fh, axh = plt.subplots(4,4, figsize=(10,10))

for i in range(4):
    for j in range(4):
        # Construct that Haar basis and display it
        Bij = np.outer(H[i,:], H[j,:])
        axh[i][j].imshow(Bij, cmap='gray', vmin=-1, vmax=1)
        axh[i][j].axes.get_xaxis().set_visible(False)
        axh[i][j].axes.get_yaxis().set_visible(False)

#plt.suptitle('Haar Basis')
plt.show()

fe.tight_layout() #minimize padding for slightly better visual.
fh.tight_layout()

#https://stackoverflow.com/questions/5812960/change-figure-window-title-in-pylab
fe.canvas.set_window_title('Euclidean/Standard Basis')
fh.canvas.set_window_title('Haar Basis')


#####################################################################


#So now, a simple color image. F is "F", CF is a random color version.
F = np.array([[0, 1, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1], [0, 1, 0, 0]])
CF = np.concatenate([np.expand_dims(F*np.random.rand(4,4), axis=2)
                     for x in range(3)], axis=2)

#the transform image: We decompose the image according to
#H by T = H*F*H', where H' is H.transpose(). T is a 4x4
#of the transform coefficients.
T = np.matmul(H, np.matmul(F, H.transpose()))

#The reconstructed image: Properties of the orthonormal
#basis make reversing the transform as easy as applying
#H and H' in a different order: F = H'*T*H
TR = np.matmul(H.transpose(), np.matmul(T, H))

f, ax = plt.subplots(1,3, figsize=(12,3))

ax[0].imshow(F, cmap='gray')
ax[0].set_title('Image')

ax[1].imshow(T, cmap='gray', vmin=T.min(), vmax=T.max())
ax[1].set_title('Transform Coefficients')

ax[2].imshow(TR, cmap='gray')
ax[2].set_title('Reconstruction')

f.tight_layout()
f.canvas.set_window_title('Image, Transform, and Reconstruction')



#See DIP 6.9.
def makeHaarMatrix(size=4):
    if not np.log2(size).is_integer():
        raise ValueError('makeHaarMatrix: input must be power of 2 (%d).' % size)

    H = np.zeros((size,size))
    H[0,:] = 1.
    # See the print statements for what took a while to debug.
    for u in range(1, size):
        # print('working row ' + str(u))
        mag = 2 ** np.floor(np.log2(u)).astype('int')
        howmany = size//(2*mag)
        # print('\tmag is %d, howmany is %d...' % (mag, howmany))


        row = howmany*[np.sqrt(mag)] + howmany*[-np.sqrt(mag)] + (size-2*howmany)*[0]
        # print('\trow to roll is ' + str(row))
        # print('\twill roll by ' + str(u%mag))

        nrow = np.array(row)

        H[u,:] = np.roll(nrow, 2*howmany*(u%mag))

    H = H/np.sqrt(size)

    return H

#Make random basis until it succeeds.
def makeRandomBasis(size=4):
    BP = np.random.rand(size,size)
    BN = orth(BP)
    if (BN.shape[1] != size):
        return makeRandomBasis(size)
    return BN

"""
Karhunen Loeve Transform version: basically principal components on the
image samples, would give the "best" representation for the provided
image. To be compatible with the wavelet decomposition examples that 
use this module, I'm returning the size x size sort-of transform matrix:
that is, I'm doing PCA on the size x 1 samples in the image and not the 
size x size patches in the image.
Assumes both sides of the provided image are divisible by size.
http://nbviewer.jupyter.org/github/sukhbinder/Notebooks/blob/master/Karhunen%20Loeve%20Transform.ipynb
https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html
https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html
"""
def makeKLTBasis(I, size=4):
    Ishape = I.shape
    #Get size x whatever samples.
    colSamples = np.reshape(I, (size, -1), order='F')
    if (len(I.shape) == 3):
        rowSamples = np.reshape(I.transpose((1, 0, 2)), (size,-1), order='F')
    else:
        rowSamples = np.reshape(I.transpose(), (size, -1), order='F')
    allSamples = np.concatenate((colSamples, rowSamples), axis = 1)

    # every column of allSamples represents an observation.
    val, vec = np.linalg.eig(np.cov(allSamples, rowvar=True))

    # Make sure the eigenvectors are sorted in decreasing order of importance.
    vecsorted = vec[:, np.argsort(val)[-1::-1]]

    return vecsorted.T


def makeDCTMatrix(size=4):
    D = np.zeros((size, size))
    x = np.arange(size)
    D[0,:] = np.sqrt(1/size)
    for u in range(1,size):
        D[u,:] = np.sqrt(2/size)*np.cos(((2*x+1)*u*np.pi)/(2*size))
    return D

# For completeness
def makeStandardMatrix(size=4):
    return np.eye(size)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
"""
Joshua Stough
DIP

Demo showing reconstruction with standard vs dct basis over the whole image.
Just like old happyFace demo in matlab that showed the basis vectors and the
reconstruction so far.
"""

import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation

from waveletUtil import *

# parameters for the script
IMAGEFILE = 'happy128.png'
SNAME = 'Standard'



I = plt.imread(IMAGEFILE)


ishape = I.shape

if len(ishape) < 3:
    raise ValueError('reconstructionCompressionAnimation: expecting color image')

if ishape[0] != ishape[1]:
    print('reconstructionCompressionAnimation: squaring the image')
    minn = min(ishape[:2])
    I = I[:minn, :minn, :]
    ishape = I.shape

# Make sure the data is in 0-1, so the floating point imshow is okay
I = I/I.max()

# Now the idea is to reconstruct an image one coefficient at a time
# in different transform spaces, S and H
if SNAME == 'Haar':
    S = makeHaarMatrix(ishape[0])
else:
    S = makeStandardMatrix(ishape[0])

# H = makeHaarMatrix(ishape[0])
H = makeDCTMatrix(ishape[0])

# The transform images in S and H
TI_S = np.zeros(ishape)
TI_H = np.zeros(ishape)

for chan in range(3):
    TI_S[..., chan] = np.matmul(S, np.matmul(I[..., chan], S.transpose()))
    TI_H[..., chan] = np.matmul(H, np.matmul(I[..., chan], H.transpose()))


# The reconstruction will go in different orders for the two basis sets,
# one by english-reading, the other by magnitude of the coefficient.
xs = np.meshgrid(np.arange(ishape[0]), np.arange(ishape[0]), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)
# dists = np.sum(coords*coords, axis=1) # to use distance from top-left
# dargS = np.argsort(dists) # sorts in increasing order
dargS = list(range(len(coords)))

# If you're not using the standard basis and you want to order
# the reconstruction by coefficient magnitude, do this:
if SNAME == 'Haar':
    mags = TI_S[...,0].ravel(order='F')
    dargS = np.argsort(np.abs(mags))
    dargS = list(reversed(dargS))


# just pic one of the color channels to use.
mags = TI_H[...,0].ravel(order='F')
dargH = np.argsort(np.abs(mags))
dargH = list(reversed(dargH))

# frame number
fn = 0



# The four images we're going to update are the basis images Sij and Hij,
# and the reconstructions so far SRI and HRI. This is just the initialization.
# aSij, etc. are the artists for the animation.

Sij = np.zeros((ishape[0], ishape[0]))
Hij = np.zeros((ishape[0], ishape[0]))
SRI = np.zeros(ishape)
HRI = np.zeros(ishape)


f, ax = plt.subplots(2, 2, figsize=(8,8), sharex=True, sharey=True)
f.canvas.set_window_title('Image Reconstruction with Cosine Patterns')
plt.tight_layout()

aSij = ax[0][0].imshow(Sij, cmap='gray', animated=True)
ax[0][0].set_title('%s Pattern' % SNAME)

aHij = ax[0][1].imshow(Hij, cmap='gray', animated=True)
ax[0][1].set_title('Cosine Pattern')

aSRI = ax[1][0].imshow(SRI, animated=True)
ax[1][0].set_title('%s Reconstruction' % SNAME)

aHRI = ax[1][1].imshow(HRI, animated=True)
ax[1][1].set_title('Cosine Reconstruction')

# Let's add an animated text field for the frame number.
aFNText = ax[0][0].text(np.round(.7*ishape[0]), np.round(.9*ishape[0]), 'frame %04d' % fn,
                        color='cyan', animated=True, bbox=dict(facecolor='red', alpha=0.5))


# And needed to avoid extra whitespace
# See: https://stackoverflow.com/questions/15077364/matplotlib-pyplot-imshow-removing-white-space-within-plots-when-using-attribute/
# and: https://github.com/matplotlib/matplotlib/pull/10033
for i in range(4):
    ax[i//2][i%2].set_adjustable('box')



#Now time for the animation function

# Needs to update the array data and the texts, then
# return the artists. see:
# https://matplotlib.org/api/_as_gen/matplotlib.animation.FuncAnimation.html
#
def updateFig(*args):
    global Sij, Hij, SRI, HRI, dargS, dargH, fn, TI_S, TI_H

    # update the Sij basis and add into the SRI reconstruction

    # Need j, i for coefficient mag.
    # j, i = coords[dargS[fn]]  # Get the i, j for the frame number.
    i, j = coords[dargS[fn]]
    Sij = np.outer(S[i, :], S[j, :])

    for chan in range(3):
        SRI[..., chan] += TI_S[i, j, chan] * Sij

    # Just to prove the coefficients are sorted in decreasing magnitude.
    # print('%6.3f' % TI_S[i, j, 0])

    # similarly update HRI
    j, i = coords[dargH[fn]]
    Hij = np.outer(H[i, :], H[j, :])

    for chan in range(3):
        HRI[..., chan] += TI_H[i, j, chan] * Hij

    # Just to prove the coefficients are sorted in decreasing magnitude.
    # print('%6.3f' % TI_H[i, j, 0])


    # Update the frame number fn for next time.
    fn += 1
    if (fn >= Sij.size): # just ishape[0]*ishape[0], but why keep typing that...
        fn = 0
        SRI.fill(0)
        HRI.fill(0)

    # Now with all the images updated, update the artists and return them.

    # Not sure why the single-channel basis images don't show without the
    # the clim stuff.
    aSij.set_array(Sij)
    aSij.set_clim(Sij.min(), Sij.max())
    # aSij.set_clip_on(True)

    aHij.set_array(Hij)
    aHij.set_clim(Hij.min(), Hij.max())
    # aHij.set_clip_on(True)

    aSRI.set_array(SRI.clip(0,1))
    aHRI.set_array(HRI.clip(0,1))

    aFNText.set_text('frame %04d' % (fn-1))

    return aSij, aHij, aSRI, aHRI, aFNText,


ani = animation.FuncAnimation(f, updateFig, interval=200, blit=True, repeat=True)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
sq2 = (1/2**.5)

In [17]:
(115 + -119) * sq2

-2.82842712474619

84.14570696119915